# Kaggle Competition 

In [37]:
import numpy as np
import pandas as pd
import polars as pl
import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

# Load train/val data

In [38]:
train_data = pd.read_parquet("data/train_data.parquet")

In [39]:
train_data.shape

(11774752, 101)

In [40]:
val_data = pd.read_parquet("data/validate_data.parquet")

In [41]:
val_data.shape

(639848, 101)

# define parameter

In [42]:
# feature selection ## select every feature except 0-4 
# feature_cols = ["symbol_id", "time_id"] + [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]
feature_cols = [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]

#select target values
target_cols = ["responder_6"]

# select the weight values
weight_cols = ["weight"]

In [ ]:
def validate_n_fold(model,X_val, y_val,w_val, n_folds):
    global ScaleY
    batch_size = len(y_val)//n_folds    
    for i in range(0, len(X_val), batch_size):
        X_batch = X_val[i:i+batch_size]
        y_batch = y_val[i:i+batch_size]
        w_batch = w_val[i:i+batch_size]
        preds = model.predict(X_batch)
        pred_inverse = ScaleY.inverse_transform(preds.reshape(-1, 1))
        score1 = r2_score(y_batch, pred_inverse ,sample_weight= w_batch)
        print(f'Batch {i//batch_size} r2_score: {score1:.6f}')

# data scaling

In [43]:
from sklearn.preprocessing import MinMaxScaler

ScaleX =    joblib.load("model/XGBoost_scaleX.pkl")
ScaleY =    joblib.load("model/XGBoost_scaleY.pkl")

In [44]:
# the chunk requir maximum memory
X_train =   ScaleX.transform(train_data[feature_cols])
y_train =   ScaleY.transform(train_data[target_cols])
w_train =   train_data[weight_cols]
X_val =     ScaleX.transform(val_data[feature_cols])    
y_val =     val_data[target_cols]
w_val =     val_data[weight_cols]

# import model

In [ ]:
model1 =    joblib.load("model/XGBoost_best_model.pkl")     # +.0056
model2 =    joblib.load("model/XGBoost_test.pkl")           # -.0027
model3 =    joblib.load("model/XGBoost_test2.pkl")          # +.0048
model4 =    joblib.load("model/XGBoost_test3.pkl")          # +.0056
model5 =    joblib.load("model/XGBoost_test4.pkl")          # +.0031
model6 =    joblib.load("model/XGBoost_test5.pkl")          # +.0036
model7 =    joblib.load("model/XGBoost_test6.pkl")          # +.0055
model8 =    joblib.load("model/XGBoost_test7.pkl")          # -.0039

# model ensemble